#SPRINT_2(IMAGE_UPLOADING)

#i)**Data Collection**
1.   Data augmentation structure creation
2.   Augmenting the data

#ii)**Loading data and Preprocessing**


1.   Make A List Of Paths To All Folders Where we Have Data
2.   Loading Images Into Machine Understandable Data
3.   Data Splitting Into Train And Test





#i)**Data Collection**
1.   Data augmentation structure creation
2.   Augmenting the data

In [5]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
import csv
from os import listdir
import time
from google.colab import drive
drive.mount('/content/drive')
# Nicely formatted time string to make a note of how much time it takes for augmentation 
def hms_string (sec_elapsed):
  h = int(sec_elapsed / (60 * 60))
  m = int((sec_elapsed % (60 * 60)) / 60) 
  S = sec_elapsed % 60
  return "{n}: {m}:{round(s,1)}"

def augment_data(file_dir, n_generated_samples, save_to_dir):
  """
  Arguments: file dir: A string representing the directory where images that we want to augment are found. 
  n generated samples: A string representing the number of generated samples using the given image.
  save to dir: A string representing the directory in which the generated images will be saved.
  """
#from keras.preprocessing.image import ImageDataGenerator 
#from os import listdir
  data_gen = ImageDataGenerator (rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.15,
                               shear_range=0.25,
                               zoom_range = 0.2,
                               horizontal_flip=True,
                               vertical_flip=False,
                               fill_mode='nearest', 
                               brightness_range=(0.5,1.2)
                               )  
  for filename in listdir(file_dir):
      #load the image
      image = cv2.imread(file_dir + '/' + filename) 
      # reshape the image 
      image=image.reshape((1,)+image.shape)
      # prefix of the names for the generated sampels. 
      save_prefix = 'aug_' + filename [:-4]
      # generate 'n_generated_samples' sample images 
      i=0
      for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir, 
                                 save_prefix=save_prefix, save_format='jpg'):
        i += 1
        if i > n_generated_samples: 
           break

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
start_time = time.time()
augmented_data_path = '/content/drive/MyDrive/project/augmented data'
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Bird/Great Indian Bustard Bird', n_generated_samples=8, save_to_dir=augmented_data_path+'/Bird/GIB_AUG')
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Bird/Spoon Billed Sandpiper Bird', n_generated_samples=8, save_to_dir=augmented_data_path+'/Bird/SPS_AUG')
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Flower/Corpse Flower', n_generated_samples=8, save_to_dir=augmented_data_path+'/Flowers/Corpse_AUG')
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Flower/Lady Slipper Orchid Flower', n_generated_samples=8, save_to_dir=augmented_data_path+'/Flowers/LS_Orchid_AUG')
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Mammal/Pangolin Mammal', n_generated_samples=8, save_to_dir=augmented_data_path+'/Mammal/Pangolin_AUG')
augment_data(file_dir='/content/drive/MyDrive/project/Digital Naturalist Dataset/Mammal/Senenca White Deer Mammal', n_generated_samples=8, save_to_dir=augmented_data_path+'/Mammal/SW_Deer_AUG')

end_time = time.time()
execution_time = (end_time - start_time)
print("Elapsed Time : "+str(execution_time))

Elapsed Time : 85.18387627601624


In [7]:
#Setting path to our dataset folder
import os
dirName = '/content/drive/MyDrive/project/augmented data'
folders = listdir(dirName)
#Getting the names for all the folders containing data
def getListOfFiles (dirName):
# create a list of sub directories and files (if any) 
# names in the given directory
    listOfFile = os.listdir (dirName) 
    allFiles = list()
    for fol_name in listOfFile:
        fullPath = os.path.join(dirName, fol_name) 
        allFiles.append(fullPath)
    return allFiles
Folders = getListOfFiles (dirName) 
len (Folders)
subfolders = []
for num in range(len(Folders)):
    sub_fols = getListOfFiles (Folders [num]) 
    subfolders+=sub_fols
#Now, the subfolders contains the address to all our data folders for each class 
subfolders

['/content/drive/MyDrive/project/augmented data/Bird/GIB_AUG',
 '/content/drive/MyDrive/project/augmented data/Bird/SPS_AUG',
 '/content/drive/MyDrive/project/augmented data/Flowers/Corpse_AUG',
 '/content/drive/MyDrive/project/augmented data/Flowers/LS_Orchid_AUG',
 '/content/drive/MyDrive/project/augmented data/Mammal/Pangolin_AUG',
 '/content/drive/MyDrive/project/augmented data/Mammal/SW_Deer_AUG']

#ii)Loading data and Preprocessing
1.   Make A List Of Paths To All Folders Where we Have Data
2.   Loading Images Into Machine Understandable Data
3.   Data Splitting Into Train And Test

In [8]:
#For matrix calculations and data Managememnt
import numpy as np
#Importing libraries required for the model
import tensorflow as tf
import keras
import keras.backend as K
from keras.optimizers import SGD, Adam, Adagrad, RMSprop
from keras.applications import *
from keras.preprocessing import * 
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array,load_img
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras. models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, BatchNormalization,Dropout
from keras.utils.np_utils import to_categorical 
from sklearn.model_selection import train_test_split
#For plotting charts used for data visualizations
import matplotlib.pyplot as plt
#Libraries for Locating and loading data
import glob
from PIL import Image
import os
from os import listdir

In [9]:
#Loading the data and pre processing it to make it in trainable format #
#x data will includes the data generated for each image 
#Y data will include a id no, unique for every different species, so are having 6 classes 
#there for we will get 6 ids = [0,1,2,3,4,5) 
#That will be tha label we're classifying. 
X_data = [] 
Y_data = [] 
id_no=0
#to make a list of tuples, where we'll store the info about the image, category and species 
found = []
#itering in all folders under Augmented data folder 
for paths in subfolders:
    #setting folder path for each unique class and category 
    files = glob.glob(paths + "/*.jpg") 
    #adding tuples to the list that contain folder name and subfolder name 
    found.append((paths.split('/')[-2],paths.split('/')[-1]))
    #itering all files under the folder one by one 
    for myFile in files:
            img = Image.open(myFile)
            #img.thumbnail((width, height), Image.ANTIALIAS) # resizes image in-place keeps ratio
            img = img.resize((224,224), Image.ANTIALIAS) # resizes image without ratio
            #convert the images to numpy arrays
            img = np.array(img)
            if img.shape == ( 224, 224, 3):
                # Add the numpy image to matrix with all data
                X_data.append (img)
                Y_data.append (id_no)
    id_no+=1
print(X_data)
print(Y_data)

X=np.array(X_data)
Y=np.array(Y_data)
print("x-shape",X.shape,"y shape",Y.shape)
X = X.astype('float32')/255.0 
y_cat = to_categorical(Y_data, len(subfolders))
print("X shape",X, "y_cat shape", y_cat) 
print("X shape",X.shape,"y_cat shape", y_cat.shape)
X_train,X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2)
print("The model has " + str(len(X_train)) + " inputs")

Streaming output truncated to the last 5000 lines.
        [  7,  58,  41]],

       ...,

       [[140, 255, 255],
        [140, 255, 255],
        [140, 255, 255],
        ...,
        [ 29, 104,  74],
        [ 32, 106,  75],
        [ 34, 106,  76]],

       [[140, 255, 255],
        [140, 255, 255],
        [140, 255, 255],
        ...,
        [ 36, 113,  83],
        [ 34, 109,  79],
        [ 33, 107,  76]],

       [[140, 255, 255],
        [140, 255, 255],
        [140, 255, 255],
        ...,
        [ 45, 123,  93],
        [ 40, 115,  85],
        [ 35, 111,  79]]], dtype=uint8), array([[[ 30,  71,  57],
        [ 30,  71,  57],
        [ 30,  71,  57],
        ...,
        [ 20,  61,  47],
        [ 20,  61,  47],
        [ 20,  61,  47]],

       [[ 31,  72,  58],
        [ 31,  72,  58],
        [ 31,  72,  58],
        ...,
        [ 22,  63,  49],
        [ 22,  63,  49],
        [ 22,  63,  49]],

       [[ 32,  73,  59],
        [ 32,  73,  59],
        [ 32,  73,  